# Getting Parameter Results

This file applies all the parameter functions onto the first 4 MJD_Train data sets and saves the results into a csv file so it can be used for feature engineering in the models. 

In [6]:
# Imports
import h5py
import os
import time
import numpy as np
import pandas as pd

Importing all the parameter functions that will be used to extract the parameters.

In [3]:
# Get the base directory of the repository
repo_base_dir = os.getcwd()

# Get all HDF5 files in the data directory
data_dir = os.path.join(repo_base_dir, "data/") 
data_files = [f for f in os.listdir(str(data_dir)) if f.endswith(".hdf5")]

# Import parameter extraction functions
from parameter_functions.tdrift import tdrift
from parameter_functions.rea import rea
from parameter_functions.dcr import find_dcr
from parameter_functions.peakandtailslope import extract_peak_and_tail_slope
from parameter_functions.current_amplitude import max_amplitude
from parameter_functions.fourier_lfpr import normalized_fourier, lfpr
from parameter_functions.lq80 import LQ80
from parameter_functions.agr import area_growth_rate
from parameter_functions.inflection import inflection_points
from parameter_functions.rising_edge import rising_edge_slope

In [ ]:
# %%
start = time.time()
# Parameter extraction from all waveform data
results = []
count = 0
for file in data_files:
    with h5py.File(data_dir + file, 'r') as f:
        for wave in np.arange(f['raw_waveform'].shape[0]):
            # Needed labels
            waveform = f['raw_waveform'][wave]
            tp0 = f['tp0'][wave]
            id = f['id'][wave]

            # Parameter extraction
            tdriftVal = tdrift(waveform, tp0) # 99.9%, 50%, 10%
            reaVal = rea(waveform, tp0)
            dcrVal = find_dcr(waveform)
            peakandtailVal = extract_peak_and_tail_slope(waveform)
            max_amp = max_amplitude(waveform)
            lfprVal = lfpr(normalized_fourier(waveform, tp0)[0])
            lq80 = LQ80(waveform, tp0)
            agr = area_growth_rate(waveform)
            inflectionpts = inflection_points(waveform, tp0)
            res = rising_edge_slope(waveform, tp0)

            results.append({'id': id,
                'tdrift': tdriftVal[0],
                'tdrift50': tdriftVal[1],
                'tdrift10': tdriftVal[2],
                'rea': reaVal,
                'dcr': dcrVal,
                'peakindex': peakandtailVal[0],
                'peakvalue': peakandtailVal[1],
                'tailslope': peakandtailVal[2],
                'currentamp': max_amp,
                'lfpr': lfprVal,
                'lq80': lq80,
                'areagrowthrate': agr,
                'inflection point': inflectionpts,
                'risingedgeslope': res
            })
        df = pd.DataFrame(results)
    df.to_csv("results_MJD_Train_" + str(count) + ".csv", index=False)
    count += 1
end = time.time()
print(end - start)